In [1]:
import pandas as pd
import json

# Part 1 - Questions statistics

In [2]:
questions = pd.read_json('questions.json')
len(questions)

17092

In [3]:
df = pd.read_excel('almost_full_rechecked_integrals.xlsx')

In [4]:
df.columns

Index(['full_answer', 'tokens_used', 'final_answer_latex', 'question_id',
       'model', 'true_answer', 'question_text', 'code_execution', 'error',
       'Unnamed: 0', 'endpoint', 'non_empty', 'model_answer',
       'model_answer_type', 'numeric_comparison', 'numeric_subs_error',
       'Unnamed: 0.1'],
      dtype='object')

In [5]:
df = df.drop('Unnamed: 0', axis=1)
# raw_ans = raw_ans[raw_ans.model.apply(lambda m: m in models)]
df['endpoint'] = df.model.astype(str) + '_' + df.code_execution.astype(str)
df.head()

,full_answer,tokens_used,final_answer_latex,question_id,model,true_answer,question_text,code_execution,error,endpoint,non_empty,model_answer,model_answer_type,numeric_comparison,numeric_subs_error,Unnamed: 0.1
0,Let's analyze the integral step by step.\n\nFi...,758,-\pi \log(2),12823,gemini/gemini-2.0-flash,-pi*log(2),"Solve the following integral. Assume A,B,F are...",0.0,NaN,gemini/gemini-2.0-flash_0.0,1.0,-pi*log(2),deterministic,1.0,NaN,NaN
1,"Okay, I will solve the indefinite integral usi...",533,\frac{61\left(9 - 11\cos^{2}{\left(33x \right)...,15299,gemini/gemini-2.0-flash,61*(9*sec(33*x)**2 - 11)*sec(33*x)**9/3267,Evaluate the indefinite integral. \n\n$\int \...,1.0,NaN,gemini/gemini-2.0-flash_1.0,1.0,(549 - 671*cos(33*x)**2)/(3267*cos(33*x)**11),deterministic,0.0,NaN,NaN
2,To evaluate the iterated integral \(\int_0^1dy...,1645,\frac{59 (74 e^{75} + 1)}{5625},16009,Qwen/Qwen2.5-72B-Instruct,(-1 + (1653 + 122248*exp(75))*exp(78750))*exp(...,Evaluate the iterated integral $\int_0^1dy\int...,NaN,NaN,Qwen/Qwen2.5-72B-Instruct_nan,1.0,59/5625 + 4366*exp(75)/5625,deterministic,1.0,NaN,NaN
3,Let the given integral be\n$$ I = \int \frac{ ...,2011,\frac{1}{\left(3 \left(\frac{\tan{\left(x \rig...,2152,gemini/gemma-3-27b-it,5*x**(4/5)/4 + 5*atan(x**(1/5)),"Solve the following integral. Assume A,B,F,G a...",NaN,NaN,gemini/gemma-3-27b-it_nan,1.0,(3*(tan(x) + tan(x*(G - 1)))/((-tan(x)*tan(x*(...,deterministic,0.0,NaN,NaN
4,Let the integral be $I$. We have\n$$ I = \int ...,1185,\frac{1}{2 \left(3 + x^{3}\right)^{\frac{2}{3}}},2322,gemini/gemini-2.0-flash,5*x**(4/5)/4 + 5*atan(x**(1/5)),"Solve the following integral. Assume A,B,F,G a...",0.0,NaN,gemini/gemini-2.0-flash_0.0,1.0,1/(2*(x**3 + 3)**(2/3)),deterministic,0.0,NaN,NaN


In [6]:
df['non_empty'] = ~df.full_answer.isna()
df['extracted_latex'] = ~df.final_answer_latex.isna()
df['extracted_sp'] = ~df.model_answer.isna()
df['completed_numeric'] = ~df.numeric_comparison.isna()

In [7]:
stats = df.groupby('endpoint').agg(
    n_answers=('question_id','nunique'),
    n_entries=('question_id','count'),
    n_non_empty=('non_empty','sum'),
    n_extracted_latex=('extracted_latex','sum'),
    n_extracted_sp=('extracted_sp','sum'),
    n_non_error_comparission=('completed_numeric','sum'),
    n_numeric_correct=('numeric_comparison','sum'),
)
stats

,n_answers,n_entries,n_non_empty,n_extracted_latex,n_extracted_sp,n_non_error_comparission,n_numeric_correct
endpoint,,,,,,,
DeepSeek-Prover-V2-671B_nan,14623,14624,14624,14176,13661,12908,3483.0
DeepSeek-R1_nan,15675,15677,15677,15657,15649,14627,6157.0
DeepSeek-V3_nan,16715,16718,16718,16590,16079,15356,3442.0
Qwen/Qwen2.5-72B-Instruct_nan,16590,16591,16591,16590,16169,15375,2650.0
gemini/gemini-2.0-flash_0.0,16847,16847,16847,16766,16722,15668,4182.0
gemini/gemini-2.0-flash_1.0,16041,16044,16044,15682,15319,14320,4422.0
gemini/gemini-2.5-flash-preview-04-17_0.0,16848,16850,16850,16822,16782,15715,11672.0
gemini/gemini-2.5-flash-preview-04-17_1.0,16521,16522,16522,14012,13954,12919,10355.0
gemini/gemma-3-27b-it_nan,16555,16555,16555,16526,16318,15419,2307.0


In [8]:
stats['%latex from valid ans'] = stats.n_extracted_latex / stats.n_non_empty
stats['%sp from latex'] = stats.n_extracted_sp / stats.n_extracted_latex
stats['%comparission from sp'] = stats.n_non_error_comparission / stats.n_extracted_sp

In [9]:
stats

,n_answers,n_entries,n_non_empty,n_extracted_latex,n_extracted_sp,n_non_error_comparission,n_numeric_correct,%latex from valid ans,%sp from latex,%comparission from sp
endpoint,,,,,,,,,,
DeepSeek-Prover-V2-671B_nan,14623,14624,14624,14176,13661,12908,3483.0,0.969365,0.963671,0.944880
DeepSeek-R1_nan,15675,15677,15677,15657,15649,14627,6157.0,0.998724,0.999489,0.934692
DeepSeek-V3_nan,16715,16718,16718,16590,16079,15356,3442.0,0.992344,0.969198,0.955035
Qwen/Qwen2.5-72B-Instruct_nan,16590,16591,16591,16590,16169,15375,2650.0,0.999940,0.974623,0.950894
gemini/gemini-2.0-flash_0.0,16847,16847,16847,16766,16722,15668,4182.0,0.995192,0.997376,0.936969
gemini/gemini-2.0-flash_1.0,16041,16044,16044,15682,15319,14320,4422.0,0.977437,0.976852,0.934787
gemini/gemini-2.5-flash-preview-04-17_0.0,16848,16850,16850,16822,16782,15715,11672.0,0.998338,0.997622,0.936420
gemini/gemini-2.5-flash-preview-04-17_1.0,16521,16522,16522,14012,13954,12919,10355.0,0.848081,0.995861,0.925828
gemini/gemma-3-27b-it_nan,16555,16555,16555,16526,16318,15419,2307.0,0.998248,0.987414,0.944907


In [10]:
df['extracted_sp'] = ~df.model_answer.isna()
df['completed_numeric'] = ~df.numeric_comparison.isna()

In [12]:
# extract "bugs" - valid SP but failed to compare
drop_of_mystery = df[(~df['completed_numeric']) & df['extracted_sp']]
valid_cases = df[~((~df['completed_numeric']) & df['extracted_sp'])]

In [13]:
# The cases left without a comparission value failed because there is probably no latex in the answer,
# or that the latex crappy and cannot be converted to sympy. 
# We assume that these cases are wrong
valid_cases[valid_cases.numeric_comparison.isna()] = 0.0

C:\Users\admin2\AppData\Local\Temp\ipykernel_6000\1475618760.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_cases[valid_cases.numeric_comparison.isna()] = 0.0
C:\Users\admin2\AppData\Local\Temp\ipykernel_6000\1475618760.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.0' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  valid_cases[valid_cases.numeric_comparison.isna()] = 0.0
C:\Users\admin2\AppData\Local\Temp\ipykernel_6000\1475618760.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.0' has dtype incompatible with bool, pleas

In [14]:
valid_cases = valid_cases[valid_cases['endpoint'] != 0.0]

In [15]:
valid_cases.groupby('endpoint').numeric_comparison.sum()

endpoint
DeepSeek-Prover-V2-671B_nan                       3483.0
DeepSeek-R1_nan                                   6157.0
DeepSeek-V3_nan                                   3442.0
Qwen/Qwen2.5-72B-Instruct_nan                     2650.0
gemini/gemini-2.0-flash_0.0                       4182.0
gemini/gemini-2.0-flash_1.0                       4422.0
gemini/gemini-2.5-flash-preview-04-17_0.0        11672.0
gemini/gemini-2.5-flash-preview-04-17_1.0        10355.0
gemini/gemma-3-27b-it_nan                         2307.0
gpt-4.1_0.0                                       2975.0
gpt-4.1_1.0                                       1603.0
gpt-4o-mini_0.0                                   1542.0
gpt-4o-mini_1.0                                    957.0
gpt-4o_0.0                                        1955.0
gpt-4o_1.0                                        1363.0
meta-llama/Llama-4-Scout-17B-16E-Instruct_nan     2070.0
Name: numeric_comparison, dtype: float64

In [16]:
merged = pd.merge(
    left=valid_cases,
    right=questions,
    left_on='question_id',
    right_on='Index'
)
# merged = merged[[
    #'question_id', 'model', 'code_execution', 'final_answer_latex', 'model_answer', 
    #'true_answer','model_answer_type', 'symbolic_comparison', 'numeric_comparison',
    #'Variation', 'Source']]
merged.Variation = merged.Variation.apply(str.strip)
merged.head()

,full_answer,tokens_used,final_answer_latex,question_id,model,true_answer,question_text,code_execution,error,endpoint,...,Unnamed: 0.1,extracted_latex,extracted_sp,completed_numeric,Index,Challenge,Answer in Latex,Answer in Sympy,Variation,Source
0,Let's analyze the integral step by step.\n\nFi...,758,-\pi \log(2),12823,gemini/gemini-2.0-flash,-pi*log(2),"Solve the following integral. Assume A,B,F are...",0.0,NaN,gemini/gemini-2.0-flash_0.0,...,NaN,True,True,True,12823,"Solve the following integral. Assume A,B,F are...",,-pi*log(2),Equivalence-All-Easy,OBMU 2019 - Q22
1,"Okay, I will solve the indefinite integral usi...",533,\frac{61\left(9 - 11\cos^{2}{\left(33x \right)...,15299,gemini/gemini-2.0-flash,61*(9*sec(33*x)**2 - 11)*sec(33*x)**9/3267,Evaluate the indefinite integral. \n\n$\int \...,1.0,NaN,gemini/gemini-2.0-flash_1.0,...,NaN,True,True,True,15299,Evaluate the indefinite integral. \n\n$\int \...,,33**(-1)*sec(x*33)**9*(-11 + 9*sec(x*33)**2)*6...,Numeric-All-2-S,UGMathBench\nCalculus_-_single_variable_0604
2,To evaluate the iterated integral \(\int_0^1dy...,1645,\frac{59 (74 e^{75} + 1)}{5625},16009,Qwen/Qwen2.5-72B-Instruct,(-1 + (1653 + 122248*exp(75))*exp(78750))*exp(...,Evaluate the iterated integral $\int_0^1dy\int...,NaN,NaN,Qwen/Qwen2.5-72B-Instruct_nan,...,NaN,True,True,True,16009,Evaluate the iterated integral $\int_0^1dy\int...,,75**(-2)*e**(-75**2*14)*14**(-1)*(-1 + e**(75*...,Numeric-All-2-S,MathOdyssey\nProblem 336 from Calculus and Ana...
3,Let the given integral be\n$$ I = \int \frac{ ...,2011,\frac{1}{\left(3 \left(\frac{\tan{\left(x \rig...,2152,gemini/gemma-3-27b-it,5*x**(4/5)/4 + 5*atan(x**(1/5)),"Solve the following integral. Assume A,B,F,G a...",NaN,NaN,gemini/gemma-3-27b-it_nan,...,NaN,True,True,True,2152,"Solve the following integral. Assume A,B,F,G a...",,5*x**(4/5)/4 + 5*atan(x**(1/5)),Equivalence-All-Hard,U-Math\nintegral_calc\n126c4165-b3d5-4470-8412...
4,Let the integral be $I$. We have\n$$ I = \int ...,1185,\frac{1}{2 \left(3 + x^{3}\right)^{\frac{2}{3}}},2322,gemini/gemini-2.0-flash,5*x**(4/5)/4 + 5*atan(x**(1/5)),"Solve the following integral. Assume A,B,F,G a...",0.0,NaN,gemini/gemini-2.0-flash_0.0,...,NaN,True,True,True,2322,"Solve the following integral. Assume A,B,F,G a...",,5*x**(4/5)/4 + 5*atan(x**(1/5)),Equivalence-All-Hard,U-Math\nintegral_calc\n126c4165-b3d5-4470-8412...


In [17]:
merged.to_excel('all_checked_numer_only_some_holes_with_vars.xlsx')

In [45]:
stats = merged.groupby('endpoint').agg(
    n_answers=('question_id','nunique'),
    n_correct=('numeric_comparison', 'sum')
)
stats['% correct'] = stats.n_correct / stats.n_answers
stats

,n_answers,n_correct,% correct
endpoint,,,
DeepSeek-Prover-V2-671B_nan,12908,3483.0,0.269833
DeepSeek-R1_nan,14627,6157.0,0.420934
DeepSeek-V3_nan,15356,3442.0,0.224147
Qwen/Qwen2.5-72B-Instruct_nan,15375,2650.0,0.172358
gemini/gemini-2.0-flash_0.0,15668,4182.0,0.266913
gemini/gemini-2.0-flash_1.0,14320,4422.0,0.308799
gemini/gemini-2.5-flash-preview-04-17_0.0,15715,11672.0,0.742730
gemini/gemini-2.5-flash-preview-04-17_1.0,12919,10355.0,0.801533
gemini/gemma-3-27b-it_nan,15419,2307.0,0.149621


# Manual overview at things

In [52]:
q_wise_df = pd.merge(
    left=df[df.code_execution == 0.0],
    right=df[df.code_execution == 1.0],
    on=('question_id', 'model'),
    suffixes=('_code_off', '_code_on')
)

In [63]:
weird_data = q_wise_df[
    (~q_wise_df.numeric_comparison_code_off.isna()) & 
    (q_wise_df.numeric_comparison_code_off != q_wise_df.numeric_comparison_code_on) &
    (q_wise_df.model == 'gpt-4.1')
]
weird_data.columns

Index(['full_answer_code_off', 'tokens_used_code_off',
       'final_answer_latex_code_off', 'question_id', 'model',
       'true_answer_code_off', 'question_text_code_off',
       'code_execution_code_off', 'error_code_off', 'endpoint_code_off',
       'non_empty_code_off', 'model_answer_code_off',
       'model_answer_type_code_off', 'numeric_comparison_code_off',
       'numeric_subs_error_code_off', 'extracted_latex_code_off',
       'extracted_sp_code_off', 'completed_numeric_code_off',
       'anti_code_exec_code_off', 'full_answer_code_on', 'tokens_used_code_on',
       'final_answer_latex_code_on', 'true_answer_code_on',
       'question_text_code_on', 'code_execution_code_on', 'error_code_on',
       'endpoint_code_on', 'non_empty_code_on', 'model_answer_code_on',
       'model_answer_type_code_on', 'numeric_comparison_code_on',
       'numeric_subs_error_code_on', 'extracted_latex_code_on',
       'extracted_sp_code_on', 'completed_numeric_code_on',
       'anti_code_exec_cod

In [66]:
weird_data[[
    'question_id', 'model', 'true_answer_code_off', 'question_text_code_off',
    'full_answer_code_off', 'tokens_used_code_off', 'final_answer_latex_code_off', 
    'error_code_off',
    'model_answer_type_code_off', 
    'numeric_subs_error_code_off', 
   'full_answer_code_on', 'tokens_used_code_on', 'final_answer_latex_code_on', 
    'error_code_on',
    'model_answer_type_code_on', 
    'numeric_subs_error_code_on',
    'model_answer_code_off',
    'model_answer_code_on',
    'numeric_comparison_code_off',
    'numeric_comparison_code_on',
]]

,question_id,model,true_answer_code_off,question_text_code_off,full_answer_code_off,tokens_used_code_off,final_answer_latex_code_off,error_code_off,model_answer_type_code_off,numeric_subs_error_code_off,full_answer_code_on,tokens_used_code_on,final_answer_latex_code_on,error_code_on,model_answer_type_code_on,numeric_subs_error_code_on,model_answer_code_off,model_answer_code_on,numeric_comparison_code_off,numeric_comparison_code_on
74,12462,gpt-4.1,-log((2 + 2*E)/(2 + exp(2))),"Solve the following integral. Assume A,B,F,G a...",Let's analyze the given integral step by step:...,3325,I = \int_1^2 \frac{ e^{x} ( x - 1 ) }{ x e^{x}...,NaN,deterministic,NaN,The final answer is:\n$$\n\frac{\log \left( \f...,567,\frac{\log \left( \frac{2}{e} \right) + 1}{\lo...,NaN,deterministic,NaN,Integral((x - 1)*exp(x)/(-2*I*x**2*(exp(4*I*G*...,(log(2*exp(-1)) + 1)/log(2),0.0,1.0
129,3339,gpt-4.1,(8741816*x**3 + 2)/(254616*x*(8741816*x**3 + 3...,"Calculate the following integral. \nAssume A,B...",Let's break down the integral step by step:\n\...,1083,e^{-8},NaN,deterministic,NaN,The final answer is:\n$$\n\frac{1}{1236 \left(...,187,\frac{1}{1236 \left(3 + (206 x)^3\right)^{2/3}...,NaN,deterministic,NaN,exp(-8),C + 1/(1236*(8741816*x**3 + 3)**(2/3)),0.0,1.0
306,14828,gpt-4.1,73*(970299*x**3 + 102)/(3976814556*x*(970299*x...,Solve the following integral. \n\n$ \int \frac...,Let's simplify the integral step by step.\n\nF...,3832,\frac{219 (153 + (99x)^3)^{1/3} }{7722 \cdot 1...,NaN,deterministic,NaN,The final answer is:\n$$\n-\frac{73}{26 \cdot ...,234,-\frac{73}{26 \cdot 99^2} \int \frac{1}{x^2 \l...,NaN,deterministic,NaN,C + 803*x**2/(135252*(970299*x**3 + 153)**(2/3...,-73*Integral(1/(x**2*(970299*x**3 + 153)**(5/3...,1.0,0.0
318,15386,gpt-4.1,4680*x + 65*sin(864*x)/9 + 65*sin(1728*x)/72,Evaluate the indefinite integral.\n\n$\int 208...,Let's first simplify the constants inside the ...,689,4680x + \frac{6240\sin(864x)}{864} + \frac{156...,NaN,deterministic,NaN,The final answer is:\n$$\n130 \int \cos^4(432 ...,143,"130 \int \cos^4(432 x)\, dx",NaN,deterministic,NaN,C + 4680*x + 65*sin(864*x)/9 + 65*sin(1728*x)/72,"130*Integral(cos(432*x)**4, x)",1.0,0.0
348,10512,gpt-4.1,3*x**2/2 - 8*x*cos(x + 3) - 4*(x**2 - 2)*sin(x...,Solve the following integral. Assume the param...,Let's analyze and simplify the integral step b...,1983,\frac{1}{3} \log(x + 1) - \frac{1}{6} \log(x^2...,NaN,deterministic,NaN,The final answer is:\n$$\n\int \left( 3 \left...,688,\int \left( 3 \left(\frac{\log_A\left(\frac{x...,"Traceback (most recent call last):\n File ""C:...",error,NaN,C + log(x + 1)/3 - log(x**2 - x + 1)/6 + sqrt(...,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82849,15119,gpt-4.1,-175/144,Calculate the following limit.\n\n$ \frac{1 \...,Let's try to simplify the limit:\n\n$$\lim_{x ...,677,-\frac{7}{8} \cdot \frac{1}{2} \cdot \frac{910...,NaN,deterministic,NaN,The final answer is:\n$$\n\frac{35}{96}\n$$,181,\frac{35}{96},NaN,deterministic,NaN,-175/144,35/96,1.0,0.0
82858,1538,gpt-4.1,exp(1/9),Evaluate\n$ \lim_{x \to 0^+} \left(\frac{F \s...,Let's carefully analyze each part.\n\n### 1) S...,1960,e^{1/9},NaN,deterministic,NaN,The final answer is:\n$$\n\frac{6}{\pi^2}\n$$,421,\frac{6}{\pi^2},NaN,deterministic,NaN,exp(1/9),6/pi**2,1.0,0.0
82868,257,gpt-4.1,"Sum(n*x**n*(n**2 - 1), (n, 2, oo))/6",Find the (infinite) power series of $f(x) \cdo...,"Let's proceed step by step.\n\nFirst, let's in...",1457,\sum_{k=2}^{\infty} \frac{(k-1) k (k+1)}{6} x^k,NaN,deterministic,NaN,The final answer is:\n\n$$\n\sum_{n=2}^{\infty...,419,\sum_{n=2}^{\infty} \left( \sum_{k=1}^{n-1} k ...,NaN,deterministic,NaN,"Sum(k*x**k*(k - 1)*(k + 1)/6, (k, 2, oo))",Sum(-I*k*x**n*(-k + n)*(exp(I*B*x) - exp(-I*B*...,1.0,0.0
82889,463,gpt-4.1,sqrt(2)*x**5/240 + sqrt(2)*x**4/48 - sqrt(2)*x...,Compute the first 6 nonzero terms of the Macla...,Let's break down the function step by step and...,1384,f(x) = \frac{\sqrt{2}}{2} \left(1 + x - \frac{...,NaN,

In [59]:
weird_data.iloc[0]

full_answer_code_off           Let's analyze the given integral step by step:...
tokens_used_code_off                                                        3325
final_answer_latex_code_off    I = \int_1^2 \frac{ e^{x} ( x - 1 ) }{ x e^{x}...
question_id                                                                12462
model                                                                    gpt-4.1
true_answer_code_off                                -log((2 + 2*E)/(2 + exp(2)))
question_text_code_off         Solve the following integral. Assume A,B,F,G a...
code_execution_code_off                                                      0.0
error_code_off                                                               NaN
endpoint_code_off                                                    gpt-4.1_0.0
non_empty_code_off                                                          True
model_answer_code_off          Integral((x - 1)*exp(x)/(-2*I*x**2*(exp(4*I*G*...
model_answer_type_code_off  

In [236]:
test_df = merged[(merged.model == 'gpt-4.1') & (merged.Variation == 'Original')][[
    'question_id',  'model', 'code_execution', 'question_text', 'full_answer', 'final_answer_latex','true_answer',
    'model_answer', 'numeric_comparison'
]].sort_values('question_id')
test_df

,question_id,model,code_execution,question_text,full_answer,final_answer_latex,true_answer,model_answer,numeric_comparison
11368,1,gpt-4.1,1.0,Compute the first 5 nonzero terms (not necessa...,The final answer is:\n$$\n1 + x + \frac{x^2}{2...,1 + x + \frac{x^2}{2} + \frac{x^3}{3} + \frac{...,-x**5/15 - x**4/8 + x**2/2 + x + 1,7*x**4/24 + x**3/3 + x**2/2 + x + 1,0.0
78557,1,gpt-4.1,0.0,Compute the first 5 nonzero terms (not necessa...,Let's find the Maclaurin series for \( f(x) = ...,1 + x + \frac{x^2}{2} - \frac{x^4}{8} - \frac{...,-x**5/15 - x**4/8 + x**2/2 + x + 1,-x**5/15 - x**4/8 + x**2/2 + x + 1,1.0
101839,3,gpt-4.1,0.0,Find the (infinite) power series of $f(x) \cdo...,"First, note that both \( f(x) \) and \( g(x) \...",\sum_{k=2}^{\infty} \frac{(k-1)k(k+1)}{6} x^k,"Sum(n*x**n*(n**2 - 1), (n, 2, oo))/6","Sum(k*x**k*(k - 1)*(k + 1)/6, (k, 2, oo))",1.0
53403,5,gpt-4.1,0.0,\nCompute the first 6 nonzero terms of...,"First, note that\n\[\n\sin(x) \cos\left(\frac{...",\left(x + \frac{\pi}{4}\right) - \frac{\left(x...,sqrt(2)*(x**5 + 5*x**4 - 20*x**3 - 60*x**2 + 1...,x - (x + pi/4)**11/39916800 + (x + pi/4)**9/36...,0.0
73956,5,gpt-4.1,1.0,\nCompute the first 6 nonzero terms of...,The final answer is:\n$$\n\frac{\sqrt{2}}{2} +...,\frac{\sqrt{2}}{2} + \frac{\sqrt{2}}{2}x - \fr...,sqrt(2)*(x**5 + 5*x**4 - 20*x**3 - 60*x**2 + 1...,sqrt(2)*x**5/240 + sqrt(2)*x**4/48 - sqrt(2)*x...,1.0
...,...,...,...,...,...,...,...,...,...
139578,113,gpt-4.1,0.0,Evaluate the following hypergeometric function...,Let's evaluate the hypergeometric function:\n\...,\frac{1 + e^{-2}}{2},-1 + 3*log(3)/(2*log(E)),exp(-2)/2 + 1/2,0.0
119256,115,gpt-4.1,0.0,Evaluate the following hypergeometric function...,Let's evaluate the hypergeometric function:\n\...,\frac{\pi^2}{12},pi**2/12,pi**2/12,1.0
66669,115,gpt-4.1,1.0,Evaluate the following hypergeometric function...,The final answer is:\n$$\n2 \log(2)\n$$,2 \log(2),pi**2/12,2*log(2),0.0
17242,117,gpt-4.1,1.0,Evaluate the following hypergeometric function...,The final answer is:\n$$\n{}_3F_2\left( \begin...,"{}_3F_2\left( \begin{array}{c} -1,-1,-1 \\ -1,...",0,"hyper((-1,), (), 1)",1.0


In [237]:
test_df[test_df.question_text.apply(lambda q: '\\int' not in q)].groupby('code_execution').count()

,question_id,model,question_text,full_answer,final_answer_latex,true_answer,model_answer,numeric_comparison
code_execution,,,,,,,,
0.0,39,39,39,39,39,39,39,39
1.0,36,36,36,36,36,36,36,36


In [238]:
test_stats = test_df.groupby('question_id').agg(
    replies=('question_id', 'count'),
    correct=('numeric_comparison', 'sum'),
    # code_on_correct=('numeric_comparison', lambda x: (x * df.loc[x.index, 'code_execution']).sum()),
    # code_off_correct=('numeric_comparison', lambda x: (x * (1-df.loc[x.index, 'code_execution'])).sum()),
).reset_index().sort_values('correct', ascending=False)
test_stats

,question_id,replies,correct
24,83,2,2.0
22,77,2,2.0
18,67,2,2.0
16,63,2,2.0
13,47,2,2.0
10,41,2,2.0
7,15,2,2.0
26,87,2,2.0
25,85,2,2.0
38,117,2,2.0


In [239]:
test_df[test_df.question_id.apply(lambda q: q in test_stats[test_stats.correct == 1].question_id.values)]

,question_id,model,code_execution,question_text,full_answer,final_answer_latex,true_answer,model_answer,numeric_comparison
11368,1,gpt-4.1,1.0,Compute the first 5 nonzero terms (not necessa...,The final answer is:\n$$\n1 + x + \frac{x^2}{2...,1 + x + \frac{x^2}{2} + \frac{x^3}{3} + \frac{...,-x**5/15 - x**4/8 + x**2/2 + x + 1,7*x**4/24 + x**3/3 + x**2/2 + x + 1,0.0
78557,1,gpt-4.1,0.0,Compute the first 5 nonzero terms (not necessa...,Let's find the Maclaurin series for \( f(x) = ...,1 + x + \frac{x^2}{2} - \frac{x^4}{8} - \frac{...,-x**5/15 - x**4/8 + x**2/2 + x + 1,-x**5/15 - x**4/8 + x**2/2 + x + 1,1.0
101839,3,gpt-4.1,0.0,Find the (infinite) power series of $f(x) \cdo...,"First, note that both \( f(x) \) and \( g(x) \...",\sum_{k=2}^{\infty} \frac{(k-1)k(k+1)}{6} x^k,"Sum(n*x**n*(n**2 - 1), (n, 2, oo))/6","Sum(k*x**k*(k - 1)*(k + 1)/6, (k, 2, oo))",1.0
53403,5,gpt-4.1,0.0,\nCompute the first 6 nonzero terms of...,"First, note that\n\[\n\sin(x) \cos\left(\frac{...",\left(x + \frac{\pi}{4}\right) - \frac{\left(x...,sqrt(2)*(x**5 + 5*x**4 - 20*x**3 - 60*x**2 + 1...,x - (x + pi/4)**11/39916800 + (x + pi/4)**9/36...,0.0
73956,5,gpt-4.1,1.0,\nCompute the first 6 nonzero terms of...,The final answer is:\n$$\n\frac{\sqrt{2}}{2} +...,\frac{\sqrt{2}}{2} + \frac{\sqrt{2}}{2}x - \fr...,sqrt(2)*(x**5 + 5*x**4 - 20*x**3 - 60*x**2 + 1...,sqrt(2)*x**5/240 + sqrt(2)*x**4/48 - sqrt(2)*x...,1.0
65870,9,gpt-4.1,0.0,Compute $\lim_{x \to 0}\left(\frac{ 2 \cdot \c...,Let's analyze the limit:\n\[\n\lim_{x \to 0}\l...,\frac{1}{150},1/150,1/150,1.0
83283,9,gpt-4.1,1.0,Compute $\lim_{x \to 0}\left(\frac{ 2 \cdot \c...,The final answer is:\n$$\frac{1}{10}$$,\frac{1}{10},1/150,1/10,0.0
75259,17,gpt-4.1,1.0,Evaluate\n$ \lim_{x \to 0^{+}} \left( \left( \...,The final answer is:\n$$\ne^{2/9}\n$$,e^{2/9},exp(4/9),exp(2/9),0.0
98327,17,gpt-4.1,0.0,Evaluate\n$ \lim_{x \to 0^{+}} \left( \left( \...,Let's first analyze the limit:\n\n$$\n\lim_{x ...,e^{4/9},exp(4/9),exp(4/9),1.0
137367,19,gpt-4.1,0.0,Evaluate\n$ \lim_{x \to 0}\left( \left| \frac{...,Let's analyze the expression: \n\[\n\lim_{x \t...,e^{-1/24},exp(-1/24),exp(-1/24),1.0


In [240]:
test_row = test_df.loc[83283]
test_row

question_id                                                           9
model                                                           gpt-4.1
code_execution                                                      1.0
question_text         Compute $\lim_{x \to 0}\left(\frac{ 2 \cdot \c...
full_answer                      The final answer is:\n$$\frac{1}{10}$$
final_answer_latex                                         \frac{1}{10}
true_answer                                                       1/150
model_answer                                                       1/10
numeric_comparison                                                  0.0
Name: 83283, dtype: object

In [241]:
print(questions[questions.Index == test_row.question_id].Challenge.iloc[0])

Compute $\lim_{x \to 0}\left(\frac{ 2 \cdot \cos(x)+4 }{ 5 \cdot x^3 \cdot \sin(x) }-\frac{ 6 }{ 5 \cdot x^4 }\right)$


In [242]:
print(test_row.full_answer)

The final answer is:
$$\frac{1}{10}$$


In [137]:
print("import sympy\nfrom sympy import integrate, sin, cos, Symbol\n\nx = Symbol('x')\nintegrand = 3 / (sin(2*x)**7 * cos(2*x))\nintegral_result = integrate(integrand, x)\nprint(integral_result)\n")

import sympy
from sympy import integrate, sin, cos, Symbol

x = Symbol('x')
integrand = 3 / (sin(2*x)**7 * cos(2*x))
integral_result = integrate(integrand, x)
print(integral_result)



In [152]:
import sympy as sp

In [155]:
model_answer = sp.parse_expr(test_row.model_answer)
model_answer

(-6*sin(2*x)**4 - 3*sin(2*x)**2 - 2)/(8*sin(2*x)**6) - 3*log(sin(2*x)**2 - 1)/4 + 3*log(sin(2*x))/2

In [156]:
true_answer = sp.parse_expr(test_row.true_answer)
true_answer

C + 3*log(Abs(tan(2*x)))/2 - 9/(4*tan(2*x)**2) - 9/(8*tan(2*x)**4) - 1/(4*tan(2*x)**6)

In [158]:
for i in range(1, 6):
    print((true_answer.subs({'x': i, 'C': 0}) - model_answer.subs({'x': i})).evalf())

1.375 + 2.35619449019234*I
1.375 - 2.35619449019234*I
1.375 - 2.35619449019234*I
1.375 + 2.35619449019234*I
1.375 - 2.35619449019234*I


In [171]:
NUMER_SUBS_FILE = 'numer_subs.json'

with open(NUMER_SUBS_FILE, 'r') as f:
    numer_subs = json.load(f)

import check_answer_rowwise, importlib
check_answer_rowwise = importlib.reload(check_answer_rowwise)

In [168]:
check_answer_rowwise.compare_numeric(true_answer, model_answer, numer_subs['35'], strict=False)

True

# Redo numerical comparission

In [172]:
checked = []
for i, row in test_df.iterrows():
    print(i)
    checked.append(check_answer_rowwise.check_answers(row.to_dict(), numer_subs[str(row.question_id)], 'shitfuck.xlsx'))

100983
74709
1450
12612
27805
43164
10267
115553
114831
58247
40058
120097
54439
44717
115177
100341
86242
125173
75134
64113
1779
113447
75680
10876
35113
33704
29977
31602
4837
114979
70669
54302
120301
117794
78039
5229
15788
117448
123941
39314
42177
34419
31826
66864
70193
95782
76793
43683
79824
118039
68332
22373
11348
42326
106525
43876
96534
50146
96046
95887
101085
86634
47831
45724
90940
11023
49269
37322
119369
59887
7552
27376
114332
32045
124758
108827
27764
85144
49994
52058
56570
8753
68531
117627
102060
49386
22492
115549
16852
71076
43703
53324
82767
49376
92855
57546
114911
101480
113406
19969
40039
115207
33448
33011
32440
15851
71532
72473
119014
1443
39738
103076


In [210]:
retest_df = pd.DataFrame.from_records(checked)
retest_df.groupby('code_execution').count()

,question_id,model,question_text,full_answer,final_answer_latex,true_answer,model_answer,numeric_comparison,model_answer_type
code_execution,,,,,,,,,
0.0,57,57,57,57,57,57,57,57,57
1.0,55,55,55,55,55,55,55,55,55


In [211]:
retest_df[retest_df.question_id == 25]

,question_id,model,code_execution,question_text,full_answer,final_answer_latex,true_answer,model_answer,numeric_comparison,model_answer_type
24,25,gemini/gemini-2.5-flash-preview-04-17,0.0,Solve the integral:\n$\n\int \frac{ 1 }{ \sin(...,The final answer is:\n$$ -\frac{1}{6}\cot^6(x)...,-\frac{1}{6}\cot^6(x) - \frac{3}{4}\cot^4(x) -...,C + log(Abs(tan(x))) - 3/(2*tan(x)**2) - 3/(4*...,C + log(Abs(tan(x))) - cot(x)**6/6 - 3*cot(x)*...,False,deterministic
25,25,gemini/gemini-2.5-flash-preview-04-17,1.0,Solve the integral:\n$\n\int \frac{ 1 }{ \sin(...,"code=""from sympy import integrate, Symbol, sin...",-\frac{6\sin^4(x) + 3\sin^2(x) + 2}{12\sin^6(x...,C + log(Abs(tan(x))) - 3/(2*tan(x)**2) - 3/(4*...,-(6*sin(x)**4 + 3*sin(x)**2 + 2)/(12*sin(x)**6...,True,deterministic


In [212]:
test_stats = retest_df.groupby('question_id').agg(
    replies=('question_id', 'count'),
    correct=('numeric_comparison', 'sum'),
    # code_on_correct=('numeric_comparison', lambda x: (x * df.loc[x.index, 'code_execution']).sum()),
    # code_off_correct=('numeric_comparison', lambda x: (x * (1-df.loc[x.index, 'code_execution'])).sum()),
).reset_index().sort_values('correct', ascending=False)
test_stats

,question_id,replies,correct
0,1,2,2
1,3,2,2
2,5,2,2
3,7,2,2
4,9,2,2
5,11,2,2
6,13,2,2
7,15,2,2
8,17,2,2
9,19,2,2


In [213]:
retest_df[retest_df.question_id.apply(lambda q: q in test_stats[test_stats.correct == 1].question_id.values)]

,question_id,model,code_execution,question_text,full_answer,final_answer_latex,true_answer,model_answer,numeric_comparison,model_answer_type
24,25,gemini/gemini-2.5-flash-preview-04-17,0.0,Solve the integral:\n$\n\int \frac{ 1 }{ \sin(...,The final answer is:\n$$ -\frac{1}{6}\cot^6(x)...,-\frac{1}{6}\cot^6(x) - \frac{3}{4}\cot^4(x) -...,C + log(Abs(tan(x))) - 3/(2*tan(x)**2) - 3/(4*...,C + log(Abs(tan(x))) - cot(x)**6/6 - 3*cot(x)*...,False,deterministic
25,25,gemini/gemini-2.5-flash-preview-04-17,1.0,Solve the integral:\n$\n\int \frac{ 1 }{ \sin(...,"code=""from sympy import integrate, Symbol, sin...",-\frac{6\sin^4(x) + 3\sin^2(x) + 2}{12\sin^6(x...,C + log(Abs(tan(x))) - 3/(2*tan(x)**2) - 3/(4*...,-(6*sin(x)**4 + 3*sin(x)**2 + 2)/(12*sin(x)**6...,True,deterministic
32,35,gemini/gemini-2.5-flash-preview-04-17,1.0,Solve the integral:\n$\n\int \frac{ 3 }{ \sin(...,"code=""import sympy\nfrom sympy import integrat...",-\frac{6\sin^{4}(2x) + 3\sin^{2}(2x) + 2}{8\si...,C + 3*log(Abs(tan(2*x)))/2 - 9/(4*tan(2*x)**2)...,-(6*sin(2*x)**4 + 3*sin(2*x)**2 + 2)/(8*sin(2*...,True,deterministic
33,35,gemini/gemini-2.5-flash-preview-04-17,0.0,Solve the integral:\n$\n\int \frac{ 3 }{ \sin(...,The final answer is:\n$$-\frac{1}{4}\tan^{-6}(...,-\frac{1}{4}\tan^{-6}(2x) - \frac{9}{8}\tan^{-...,C + 3*log(Abs(tan(2*x)))/2 - 9/(4*tan(2*x)**2)...,C + 3*log(Abs(tan(2*x)))/2 - 9/(4*tan(2*x)**2)...,False,deterministic
34,37,gemini/gemini-2.5-flash-preview-04-17,1.0,Solve the integral:\n$\n\int \frac{ 1 }{ \sin(...,"code=""import sympy\n\nx = sympy.Symbol('x')\nf...",\frac{3\cos^3(8x) - 5\cos(8x)}{8(8\cos^4(8x) -...,C + 3*log(Abs(tan(4*x)))/64 + tan(4*x)**4/512 ...,C + (3*cos(8*x)**3 - 5*cos(8*x))/(64*cos(8*x)*...,True,deterministic
35,37,gemini/gemini-2.5-flash-preview-04-17,0.0,Solve the integral:\n$\n\int \frac{ 1 }{ \sin(...,The final answer is:\n$$-\frac{1}{32}\csc^3(8x...,-\frac{1}{32}\csc^3(8x)\cot(8x) - \frac{3}{64}...,C + 3*log(Abs(tan(4*x)))/64 + tan(4*x)**4/512 ...,C + 3*log(Abs(tan(4*x)))/64 - cot(8*x)*csc(8*x...,False,deterministic
77,81,gemini/gemini-2.5-flash-preview-04-17,1.0,What is the integral of $ 2x - x^7atan(3) $,"code=""import sympy\n\nx = sympy.Symbol('x')\n#...",x^2 - \frac{x^8 \tan(3)}{8},-x**8*atan(3)/8 + x**2,-x**8*tan(3)/8 + x**2,False,deterministic
78,81,gemini/gemini-2.5-flash-preview-04-17,0.0,What is the integral of $ 2x - x^7atan(3) $,The final answer is:\n$$x^2 - \frac{\arctan(3)...,x^2 - \frac{\arctan(3)x^8}{8} + C,-x**8*atan(3)/8 + x**2,C - x**8*atan(3)/8 + x**2,True,deterministic
95,105,gemini/gemini-2.5-flash-preview-04-17,0.0,Solve the following integral $\int_0^{\frac{\p...,The final answer is:\n$$\frac{\pi^2}{16}$$,\frac{\pi^2}{16},pi**2/16,pi**2/16,True,deterministic
104,115,gemini/gemini-2.5-flash-preview-04-17,0.0,Evaluate the following hypergeometric function...,The given hypergeometric function is $ {}_3F_2...,\int_0^1 \frac{1-e^{-t}}{t} dt,pi**2/12,"Integral((1 - exp(-t))/t, (t, 0, 1))",False,deterministic


In [ ]:
test_df[test_df.question_text.apply(lambda q: '\\int' not in q)].groupby('code_execution').count()

In [183]:
test_df[test_df.question_id.apply(lambda q: q in test_stats[test_stats.correct == 1].question_id.values)].head(4)

,question_id,model,code_execution,question_text,full_answer,final_answer_latex,true_answer,model_answer,numeric_comparison
35113,25,gemini/gemini-2.5-flash-preview-04-17,0.0,Solve the integral:\n$\n\int \frac{ 1 }{ \sin(...,The final answer is:\n$$ -\frac{1}{6}\cot^6(x)...,-\frac{1}{6}\cot^6(x) - \frac{3}{4}\cot^4(x) -...,C + log(Abs(tan(x))) - 3/(2*tan(x)**2) - 3/(4*...,C + log(Abs(tan(x))) - cot(x)**6/6 - 3*cot(x)*...,1.0
33704,25,gemini/gemini-2.5-flash-preview-04-17,1.0,Solve the integral:\n$\n\int \frac{ 1 }{ \sin(...,"code=""from sympy import integrate, Symbol, sin...",-\frac{6\sin^4(x) + 3\sin^2(x) + 2}{12\sin^6(x...,C + log(Abs(tan(x))) - 3/(2*tan(x)**2) - 3/(4*...,-(6*sin(x)**4 + 3*sin(x)**2 + 2)/(12*sin(x)**6...,0.0
120301,35,gemini/gemini-2.5-flash-preview-04-17,1.0,Solve the integral:\n$\n\int \frac{ 3 }{ \sin(...,"code=""import sympy\nfrom sympy import integrat...",-\frac{6\sin^{4}(2x) + 3\sin^{2}(2x) + 2}{8\si...,C + 3*log(Abs(tan(2*x)))/2 - 9/(4*tan(2*x)**2)...,-(6*sin(2*x)**4 + 3*sin(2*x)**2 + 2)/(8*sin(2*...,0.0
117794,35,gemini/gemini-2.5-flash-preview-04-17,0.0,Solve the integral:\n$\n\int \frac{ 3 }{ \sin(...,The final answer is:\n$$-\frac{1}{4}\tan^{-6}(...,-\frac{1}{4}\tan^{-6}(2x) - \frac{9}{8}\tan^{-...,C + 3*log(Abs(tan(2*x)))/2 - 9/(4*tan(2*x)**2)...,C + 3*log(Abs(tan(2*x)))/2 - 9/(4*tan(2*x)**2)...,1.0


In [207]:
check_answer_rowwise = importlib.reload(check_answer_rowwise)

In [209]:
import math_parsers
question_data = test_df.loc[35113].to_dict()
res = check_answer_rowwise.compare_numeric(
    math_parsers.parse_sympy_str(question_data['true_answer']), 
    math_parsers.parse_sympy_str(question_data['model_answer']), 
    numer_subs[str(question_data['question_id'])],
    strict=True,
    debug=True
)
res, question_data

true_answer:  C + log(Abs(tan(x))) - 3/(2*tan(x)**2) - 3/(4*tan(x)**4) - 1/(6*tan(x)**6)
model_answer:  C + log(Abs(tan(x))) - cot(x)**6/6 - 3*cot(x)**4/4 - 3*cot(x)**2/2
subs_vals: [[{'x': -2.274362004987207, 'C': 0, C: 0}, '-1.36600032391824'], [{'x': 4.002509051288649, 'C': 0, C: 0}, '-1.43210055314669'], [{'x': -13.017065426598313, 'C': 0, C: 0}, '-33.7877699297775'], [{'x': 2.3228595183535026, 'C': 0, C: 0}, '-1.93191197014401'], [{'x': -14.396611028932089, 'C': 0, C: 0}, '1.21696023017162']]
subs:  {'x': -2.274362004987207, 'C': 0, C: 0}
diff:  1.38168279232101e-15
subs:  {'x': 4.002509051288649, 'C': 0, C: 0}
diff:  1.39543374935111e-15
subs:  {'x': -13.017065426598313, 'C': 0, C: 0}
diff:  0
subs:  {'x': 2.3228595183535026, 'C': 0, C: 0}
diff:  5.74675783256521e-16
subs:  {'x': -14.396611028932089, 'C': 0, C: 0}
diff:  1.73335470994834e-15


(True,
 {'question_id': 25,
  'model': 'gemini/gemini-2.5-flash-preview-04-17',
  'code_execution': 0.0,
  'question_text': 'Solve the integral:\n$\n\\int \\frac{ 1 }{ \\sin(x)^7 \\cdot \\cos(x) }   dx\n$',
  'full_answer': 'The final answer is:\n$$ -\\frac{1}{6}\\cot^6(x) - \\frac{3}{4}\\cot^4(x) - \\frac{3}{2}\\cot^2(x) + \\ln|\\tan(x)| + C $$',
  'final_answer_latex': '-\\frac{1}{6}\\cot^6(x) - \\frac{3}{4}\\cot^4(x) - \\frac{3}{2}\\cot^2(x) + \\ln|\\tan(x)| + C',
  'true_answer': 'C + log(Abs(tan(x))) - 3/(2*tan(x)**2) - 3/(4*tan(x)**4) - 1/(6*tan(x)**6)',
  'model_answer': 'C + log(Abs(tan(x))) - cot(x)**6/6 - 3*cot(x)**4/4 - 3*cot(x)**2/2',
  'numeric_comparison': 1.0})

In [205]:
import numpy as np
diffs= [3.77475828372553e-15, 3.99680288865056e-15, 0, 2.22044604925031e-15, 4.21884749357559e-15]

mean_diff = np.mean(diffs)
allowed_diff = 1e-5
[abs((diff - mean_diff) ) for diff in diffs]


[np.float64(9.325873406851321e-16),
 np.float64(1.1546319456101626e-15),
 np.float64(2.8421709430403977e-15),
 np.float64(6.217248937900878e-16),
 np.float64(1.3766765505351924e-15)]

In [ ]:
5.74675783256521e-16
1.73335470994834e-15
1.38168279232101e-15
1.39543374935111e-15

In [188]:
question_data = test_df.loc[33704].to_dict()
check_answer_rowwise.check_answers(
    question_data, 
    numer_subs[str(question_data['question_id'])],
    f'crap.xlsx'
)

{'question_id': 25,
 'model': 'gemini/gemini-2.5-flash-preview-04-17',
 'code_execution': 1.0,
 'question_text': 'Solve the integral:\n$\n\\int \\frac{ 1 }{ \\sin(x)^7 \\cdot \\cos(x) }   dx\n$',
 'full_answer': 'code="from sympy import integrate, Symbol, sin, cos, tan, log\\n\\nx = Symbol(\'x\')\\nintegrand = 1 / (sin(x)**7 * cos(x))\\nintegral_result = integrate(integrand, x)\\nprint(integral_result)\\n" language=<Language.PYTHON: \'PYTHON\'>outcome=<Outcome.OUTCOME_OK: \'OUTCOME_OK\'> output=\'-(6*sin(x)**4 + 3*sin(x)**2 + 2)/(12*sin(x)**6) - log(sin(x)**2 - 1)/2 + log(sin(x))\\n\'The final answer is:\n$$ -\\frac{6\\sin^4(x) + 3\\sin^2(x) + 2}{12\\sin^6(x)} - \\frac{1}{2}\\log(\\sin^2(x) - 1) + \\log(\\sin(x)) $$',
 'final_answer_latex': '-\\frac{6\\sin^4(x) + 3\\sin^2(x) + 2}{12\\sin^6(x)} - \\frac{1}{2}\\log(\\sin^2(x) - 1) + \\log(\\sin(x))',
 'true_answer': 'C + log(Abs(tan(x))) - 3/(2*tan(x)**2) - 3/(4*tan(x)**4) - 1/(6*tan(x)**6)',
 'model_answer': '-(6*sin(x)**4 + 3*sin(x)**2

# cleamergedup

In [16]:
merged.columns

Index(['Unnamed: 0', 'question_id', 'model', 'code_execution', 'full_answer',
       'tokens_used', 'final_answer_latex', 'error', 'question_text',
       'true_answer', 'Unnamed: 0.1', 'model_answer', 'model_answer_type',
       'symbolic_comparison', 'numeric_comparison', 'Unnamed: 0.2', 'Index',
       'Challenge', 'Answer in Latex', 'Answer in Sympy', 'Variation',
       'Source'],
      dtype='object')

In [20]:
merged = merged.drop('Unnamed: 0.2', axis=1).drop('Unnamed: 0', axis=1).drop('Unnamed: 0.1', axis=1)

In [21]:
merged.head()

,question_id,model,code_execution,full_answer,tokens_used,final_answer_latex,error,question_text,true_answer,model_answer,model_answer_type,symbolic_comparison,numeric_comparison,Index,Challenge,Answer in Latex,Answer in Sympy,Variation,Source
0,15920,meta-llama/Llama-4-Scout-17B-16E-Instruct,NaN,## Step 1: Identify the series\nThe given seri...,889,\frac{1}{12} \tanh^{-1}\left(\left(\frac{1}{58...,NaN,Evaluate the series $\sum\limits_{n=0}^\infty ...,atanh(1/41958464026170802853330178015047089266...,atanh(1/41958464026170802853330178015047089266...,deterministic,1.0,1.0,15920,Evaluate the series $\sum\limits_{n=0}^\infty ...,,12**(-1)*atanh((29**(-1)/2)**65),Numeric-All-2-S,MathOdyssey\nProblem 328 from Calculus and Ana...
1,13327,gpt-4o-mini,0.0,To evaluate the hypergeometric function \n\n$$...,589,\frac{1}{1 + 2x}.,NaN,Evaluate the following hypergeometric function...,zoo*((-2*x - 1)*(2*x + 1) + (2*x + 1)**2)/(x**...,1/(2*x + 1),deterministic,0.0,0.0,13327,Evaluate the following hypergeometric function...,,zoo*((-2*x - 1)*(2*x + 1) + (2*x + 1)**2)/(x**...,Equivalence-One-Easy,ASyMoB\nHypergeometrics\nQ2
2,8139,gemini/gemini-2.0-flash,1.0,"code='import sympy\n\nx = sympy.Symbol(""x"")\ne...",581,\frac{3 x^{2}}{2} - 4985684 x^{2} \sin{\left(x...,NaN,What is the integral of\n$ 3 x - 4 \left(124...,pi/2,-4985684*x**2*sin(x + 3) + 3*x**2/2 - 9971368*...,deterministic,0.0,0.0,8139,Evaluate the limit\n$\lim\limits_{n\to\infty}\...,,pi/2,Equivalence-One-Easy,MathOdyssey\nProblem 329 from Calculus and Ana...
3,13636,Qwen/Qwen2.5-72B-Instruct,NaN,The hypergeometric function you provided is qu...,617,\frac{\Gamma(2) \Gamma(2) \Gamma(1) \Gamma(1)}...,NaN,Evaluate the following hypergeometric function...,pi**2/12,gamma(1 + (tan(x) + tan(x*(G - 1)))/((tan(x)*t...,deterministic,0.0,0.0,13636,Evaluate the following hypergeometric function...,,pi**2/12,Equivalence-One-Hard,ASyMoB\nHypergeometrics\nQ3
4,11749,Qwen/Qwen2.5-72B-Instruct,NaN,"To solve for \( y \) as a function of \( x \),...",1045,y = \frac{\sqrt{x}}{2},NaN,If $\log_{2 \left(- \sinh^{2}{\left(A x \righ...,1,sqrt(x)/2,deterministic,0.0,0.0,11749,If $\log_{2 \left(- \sinh^{2}{\left(A x \righ...,,1,Equivalence-All-Easy,OlympiadBench\noe_to_maths_en_comp\n2498


In [23]:
merged['unique_id'] = merged.question_id.astype(str) + '_' + merged.model.astype(str) + '_' + merged.code_execution.astype(str)

In [30]:
repeated_qs = merged.groupby('unique_id').agg(
    n=('question_id', 'count')).reset_index()
duplicate_lines = repeated_qs[repeated_qs.n > 1].merge(right=merged, on='unique_id')

In [31]:
duplicate_lines.to_excel('dups.xlsx') # Everything here was timeouts - probably didn't delete things well

In [37]:
# Drop these values
merged = merged[merged.unique_id.apply(lambda uid: uid not in duplicate_lines.unique_id.values)]

In [42]:
merged['endpoint'] = merged.model.astype(str) + '_' + merged.code_execution.astype(str)
merged.groupby('endpoint').agg(
    n=('question_id', 'count'),
    max_qid=('question_id', 'max'),
).reset_index()

C:\Users\admin2\AppData\Local\Temp\ipykernel_14796\3954524177.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged['endpoint'] = merged.model.astype(str) + '_' + merged.code_execution.astype(str)


,endpoint,n,max_qid
0,DeepSeek-Prover-V2-671B_nan,11784,17090
1,DeepSeek-R1_nan,12348,17092
2,DeepSeek-V3_nan,12962,17092
3,Qwen/Qwen2.5-72B-Instruct_nan,12822,17089
4,gemini/gemini-2.0-flash_0.0,9695,17092
5,gemini/gemini-2.0-flash_1.0,9130,17092
6,gemini/gemini-2.5-pro-preview-03-25_0.0,1184,4247
7,gemini/gemini-2.5-pro-preview-03-25_1.0,1014,4248
8,gemini/gemma-3-27b-it_nan,13107,17091
9,gpt-4.1_0.0,13597,17081


In [48]:
# Remove old openai stuff 
merged = merged[~merged.model.apply(lambda m: m.startswith('openai/'))]

In [53]:
stats = merged.groupby('endpoint').agg(
    n=('question_id', 'count'),
    max_qid=('question_id', 'max'),
    symb_correct=('symbolic_comparison', 'sum'),
    numer_correct=('numeric_comparison', 'sum'),
).reset_index()

In [54]:
stats['symb_per'] = stats.symb_correct / stats.n
stats['numer_per'] = stats.numer_correct / stats.n

In [56]:
stats.sort_values('symb_per')

,endpoint,n,max_qid,symb_correct,numer_correct,symb_per,numer_per
12,gpt-4o-mini_1.0,10376,17044,282.0,310.0,0.027178,0.029877
10,gpt-4.1_1.0,10451,16043,969.0,948.0,0.092718,0.090709
11,gpt-4o-mini_0.0,13539,17079,1549.0,1608.0,0.114410,0.118768
13,gpt-4o_0.0,9832,17079,1254.0,1342.0,0.127543,0.136493
14,gpt-4o_1.0,3180,16347,449.0,490.0,0.141195,0.154088
15,meta-llama/Llama-4-Scout-17B-16E-Instruct_nan,13045,17091,1930.0,2106.0,0.147949,0.161441
8,gemini/gemma-3-27b-it_nan,13107,17091,3183.0,3369.0,0.242847,0.257038
3,Qwen/Qwen2.5-72B-Instruct_nan,12822,17089,3534.0,3757.0,0.275620,0.293012
4,gemini/gemini-2.0-flash_0.0,9695,17092,2955.0,3056.0,0.304796,0.315214
5,gemini/gemini-2.0-flash_1.0,9130,17092,2976.0,3114.0,0.325958,0.341073


In [51]:
len(questions)

17092

In [50]:
merged.to_excel('partial_batch_checked_0805_1915.xlsx')

In [92]:
filtered_questions = pd.read_json('11_5K_questions_filtered.json')
len(filtered_questions)

10606

In [95]:
with open('11_5K_questions_subs.json', 'r') as f:
    questions_with_valid_subs = json.load(f)
len(questions_with_valid_subs)

994

In [94]:
questions_with_valid_subs

,1,2,3,4,5,6,7,8,9,10,...,1065,1066,1067,1075,1076,1077,1085,1086,1087,1088
0,"[{'x': -8.493170716946153}, 2324.3318833896583]","[{'x': 11.497369279055789, 'A': 4, 'B': 7}, 96...","[{'x': 8.121739531172317}, 226233629231.29868]","[{'A': 1, 'B': 8, 'x': 5.658044085306164, 'F':...","[{'x': 4.505229347993234}, 9.014601457957932]","[{'A': 1, 'x': 6.284291948345521, 'B': 7, 'F':...","[{'x': 2.515073638545079}, -8.456898656611]","[{'A': 3, 'x': 13.889876296726316, 'B': 3, 'F'...","[{}, 0.006666666666666]","[{'A': 5}, 0.001333333333333]",...,"[{}, 9.487735836358526]","[{}, 12512575305.609886]","[{}, 1.2874322094811831e+57]","[{}, 3.852076250063224]","[{}, 569194541611.1677]","[{}, 7.707266480970131e+69]","[{'F': 7}, 5.754602676005731]","[{'A': 9, 'F': 4}, 24.464536456131405]","[{'A': 1}, 1.284025416687741]","[{'F': 6}, 4.481689070338064]"
1,"[{'x': 4.561331906580224}, -169.77939957140902]","[{'x': -9.426408143815921, 'A': 1, 'B': 1}, 40...","[{'x': -1.398085028348084}, 3084.1019114590235]","[{'A': 7, 'B': 5, 'x': 7.630752863004656, 'F':...","[{'x': 0.573544392336799}, 0.9776804028700351]","[{'A': 9, 'x': -8.070846369651695, 'B': 1, 'F'...","[{'x': -2.493744271666639}, -2.769908497129855]","[{'A': 9, 'x': -3.448634680253427, 'B': 7, 'F'...","[{}, 0.006666666666666]","[{'A': 8}, 0.0008333333333330001]",...,"[{}, 9.487735836358526]","[{}, 12512575305.609886]","[{}, 1.2874322094811831e+57]","[{}, 3.852076250063224]","[{}, 569194541611.1677]","[{}, 7.707266480970131e+69]","[{'F': 3}, 2.117000016612675]","[{'A': 6, 'F': 8}, 44.3343365935839]","[{'A': 7}, 8.988177916814191]","[{'F': 1}, 1.284025416687741]"
2,"[{'x': -7.30996546922143}, 1054.9943910722545]","[{'x': -3.629710166286171, 'A': 9, 'B': 1}, 21...","[{'x': -3.2344841901980192}, 16862349.643547516]","[{'A': 5, 'B': 9, 'x': -6.463199931811091, 'F'...","[{'x': -24.287657171379912}, -38084.67488543853]","[{'A': 9, 'x': -12.691570978297214, 'B': 8, 'F...","[{'x': -1.739706126542366}, -0.511282840715741]","[{'A': 5, 'x': 17.916091285421437, 'B': 2, 'F'...","[{}, 0.006666666666666]","[{'A': 1}, 0.006666666666666]",...,"[{}, 9.487735836358526]","[{}, 12512575305.609886]","[{}, 1.2874322094811831e+57]","[{}, 3.852076250063224]","[{}, 569194541611.1677]","[{}, 7.707266480970131e+69]","[{'F': 8}, 7.38905609893065]","[{'A': 5, 'F': 9}, 47.43867918179263]","[{'A': 8}, 10.272203333501931]","[{'F': 8}, 7.38905609893065]"
3,"[{'x': 5.034258610218893}, -277.1511519119721]","[{'x': -0.507247474706598, 'A': 6, 'B': 6}, -2...","[{'x': -5.421451896606725}, 3189677360.6266837]","[{'A': 3, 'B': 1, 'x': -2.097828682271538, 'F'...","[{'x': 14.534646296997565}, 4711.63725640578]","[{'A': 4, 'x': -4.088827601927761, 'B': 3, 'F'...","[{'x': 5.054454983799114}, -145.76772216983696]","[{'A': 9, 'x': 6.719917755410151, 'B': 4, 'F':...","[{}, 0.006666666666666]","[{'A': 8}, 0.0008333333333330001]",...,"[{}, 9.487735836358526]","[{}, 12512575305.609886]","[{}, 1.2874322094811831e+57]","[{}, 3.852076250063224]","[{}, 569194541611.1677]","[{}, 7.707266480970131e+69]","[{'F': 3}, 2.117000016612675]","[{'A': 9, 'F': 9}, 85.38962252722673]","[{'A': 8}, 10.272203333501931]","[{'F': 3}, 2.117000016612675]"
4,"[{'x': -38.93635948862847}, 5679453.2045545215]","[{'x': -18.708373113531064, 'A': 1, 'B': 7}, -...","[{'x': 14.618883678291146}, 77400309272481.02]","[{'A': 8, 'B': 6, 'x': 12.04343770171553, 'F':...","[{'x': -1.090880067032366}, -0.299386157183386]","[{'A': 2, 'x': -4.069531067286713, 'B': 3, 'F'...","[{'x': -5.651271566738777}, -114.48391015850115]","[{'A': 6, 'x': 9.81207020673309, 'B': 7, 'F': ...","[{}, 0.006666666666666]","[{'A': 2}, 0.003333333333333]",...,"[{}, 9.487735836358526]","[{}, 12512575305.609886]","[{}, 1.2874322094811831e+57]","[{}, 3.852076250063224]","[{}, 569194541611.1677]","[{}, 7.707266480970131e+69]","[{'F': 8}, 7.38905609893065]","[{'A': 8, 'F': 6}, 35.853512562704516]","[{'A': 3}, 3.852076250063224]","[{'F': 9}, 9.487735836358526]"


In [76]:
df = pd.read_excel('checked_results.xlsx')
set(df.model.values)

{'deepseek-r1-distill-llama-70b',
 'gemini/gemini-2.0-flash',
 'gemini/gemini-2.5-pro-preview-03-25',
 'gemini/gemma-3-27b-it',
 'meta-llama/llama-4-maverick-17b-128e-instruct',
 'meta-llama/llama-4-scout-17b-16e-instruct',
 'openai/gpt-4o',
 'openai/gpt-4o-mini',
 'openai/o3-mini',
 'qwen-qwq-32b'}

In [77]:
questions = pd.read_json('11_5K_single_for_var.json')

In [78]:
df.columns, questions.columns

(Index(['full_answer', 'final_answer_latex', 'question_id', 'model',
        'true_answer', 'question_text', 'code_execution', 'model_answer',
        'model_answer_type', 'symbolic_comparison', 'numeric_comparison',
        'error'],
       dtype='object'),
 Index(['Index', 'Challenge', 'Answer in Latex', 'Answer in Sympy', 'Variation',
        'Source'],
       dtype='object'))

In [79]:
merged = pd.merge(
    left=df,
    right=questions,
    left_on='question_id',
    right_on='Index'
)
merged = merged[[
    'question_id', 'model', 'code_execution', 'final_answer_latex', 'model_answer', 
    'true_answer','model_answer_type', 'symbolic_comparison', 'numeric_comparison',
    'Variation', 'Source']]
merged.Variation = merged.Variation.apply(str.strip)

In [80]:
merged[pd.isna(merged.code_execution)]

,question_id,model,code_execution,final_answer_latex,model_answer,true_answer,model_answer_type,symbolic_comparison,numeric_comparison,Variation,Source


In [81]:
summary = merged.groupby(['model', 'code_execution', 'Variation']).agg(
    n_valid=('question_id', 'count'),
    n_symbolic_correct=('symbolic_comparison', 'sum'),
    n_numeric_correct=('numeric_comparison', 'sum'),
).reset_index()
summary.to_excel('summary.xlsx')

In [82]:
set(summary.model.values)

{'deepseek-r1-distill-llama-70b',
 'gemini/gemini-2.0-flash',
 'gemini/gemini-2.5-pro-preview-03-25',
 'gemini/gemma-3-27b-it',
 'meta-llama/llama-4-maverick-17b-128e-instruct',
 'meta-llama/llama-4-scout-17b-16e-instruct',
 'openai/gpt-4o',
 'openai/gpt-4o-mini',
 'openai/o3-mini',
 'qwen-qwq-32b'}

In [83]:
merged.groupby(['model', 'code_execution', 'Variation']).agg(
    n_valid=('question_id', 'count'),
    n_symbolic_correct=('symbolic_comparison', 'sum'),
    n_numeric_correct=('numeric_comparison', 'sum'),
)

n_valid  \
model                         code_execution Variation                        
deepseek-r1-distill-llama-70b False          Equivalence-1-Easy           1   
                                             Equivalence-Full-Easy        1   
                                             Equivalence-Full-Hard        1   
                                             Numeric-1-1                  1   
                                             Numeric-1-10                 1   
...                                                                     ...   
qwen-qwq-32b                  True           Symbolic-1                   1   
                                             Symbolic-2                   1   
                                             Symbolic-3                   1   
                                             Symbolic-4                   1   
                                             Symbolic-5                   1   

                                                                    n_symbolic_correct  \
model                         code_execution Variation                                   
deepseek-r1-distill-llama-70b False          Equivalence-1-Easy                    1.0   
                                             Equivalence-Full-Easy                 0.0   
                                             Equivalence-Full-Hard                 0.0   
                                             Numeric-1-1                           0.0   
                                             Numeric-1-10                          0.0   
...                                                                                ...   
qwen-qwq-32b                  True           Symbolic-1                            1.0   
                                             Symbolic-2                            1.0   
                                             Symbolic-3                            1.0   
                                             Symbolic-4                            0.0   
                                             Symbolic-5                            0.0   

                                                                    n_numeric_correct  
model                         code_execution Variation                                 
deepseek-r1-distill-llama-70b False          Equivalence-1-Easy                   1.0  
                                             Equivalence-Full-Easy                0.0  
                                             Equivalence-Full-Hard                0.0  
                                             Numeric-1-1                          0.0  
                                             Numeric-1-10                         0.0  
...                                                                               ...  
qwen-qwq-32b                  True           Symbolic-1                           1.0  
                                             Symbolic-2                           1.0  
                                             Symbolic-3                           1.0  
                                             Symbolic-4                           0.0  
                                             Symbolic-5                           1.0  

[372 rows x 3 columns]

# Did anyone made it?

In [87]:
merged.groupby('Variation').agg(
    n_valid=('question_id', 'count'),
    n_symbolic_correct=('symbolic_comparison', 'sum'),
    n_numeric_correct=('numeric_comparison', 'sum'),
)

,n_valid,n_symbolic_correct,n_numeric_correct
Variation,,,
Equivalence-1-Easy,20,18.0,19.0
Equivalence-Full-Easy,19,6.0,6.0
Equivalence-Full-Hard,19,7.0,11.0
Numeric-1-1,18,0.0,0.0
Numeric-1-10,18,0.0,1.0
Numeric-1-2,18,15.0,15.0
Numeric-1-3,19,10.0,11.0
Numeric-1-4,19,0.0,0.0
Numeric-1-5,18,0.0,1.0


In [88]:
merged.groupby('question_id').agg(
    n_symbolic_correct=('symbolic_comparison', 'sum'),
    n_numeric_correct=('numeric_comparison', 'sum'),
)

,n_symbolic_correct,n_numeric_correct
question_id,,
1,11.0,11.0
2,8.0,8.0
4,3.0,16.0
24,4.0,5.0
32,0.0,0.0
84,0.0,16.0
112,13.0,12.0
123,6.0,6.0
124,7.0,11.0
